<a href="https://colab.research.google.com/github/andrewli4938/TrafficLightDetection/blob/main/Group1_Traffic_Light_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from torchvision.io import read_image
import os
import glob

from ultralytics import YOLO

In [ ]:
# import our dataset
import kagglehub
path = kagglehub.dataset_download("mbornoe/lisa-traffic-light-dataset")

In [ ]:
!rm -r images_dir
!mkdir images_dir

In [ ]:
print(len(os.listdir("/content/images_dir")))

43016


In [ ]:
# centralize both annotations and images (not scattered in different folders)
annotations = pd.DataFrame()

target = "frameAnnotationsBOX.csv"
annotation_paths = glob.glob(f"{path}/**/{target}", recursive=True)

# I am going to omit the approximately 1000 pictures and labels in sample-dayClip6
# and sample-nightClip1 because it is really annoying to work with right now and and
# we have 43k other values we can work with first
for p in annotation_paths:
  tokens = p.split("/")
  if tokens[-2] == "sample-nightClip1" or tokens[-2] == "sample-dayClip6":  # omit here
    continue

  # combine the csv files into one
  new_frame = pd.read_csv(p)
  new_frame.drop(['Origin file', 'Origin frame number', 'Origin track', 'Origin track frame number'], axis=1)
  annotations = pd.concat([annotations, new_frame])

  # move all of the nested image files into /content/images_dir
  if len(tokens)==14:
    subpath = path+"/"+tokens[-3]+"/"+tokens[-3]+"/"+tokens[-2]+"/frames"
  else:
    subpath = path+"/"+tokens[-2]+"/"+tokens[-2]+"/frames"
  !cp -ri {subpath}/* /content/images_dir

filenames = annotations[annotations.columns[0]].str.split("/").str[-1]
annotations["Filename"] = filenames
annotations  # NOTE: our labels are in this df here

In [ ]:
#TODO: define custom dataset class
class TrafficLightDataset(Dataset):
  def __init__(self, annotations_df, img_dir, transform=None, target_transform=None):
    self.img_labels = annotations_df
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = read_image(img_path)
    label = tuple(self.img_labels.iloc[idx, 1:])
    if self.transform:
      image = self.transform(image)
    if self.target_transform:
      pass  # not sure how this will work on a tuple (worry about it later)
    return image, label


In [ ]:
#TODO: see if our Dataset works with a DataLoader
img_dir = "content/images_dir"
data = TrafficLightDataset(annotations, img_dir)
train_dataloader = None
test_dataloader = None


In [ ]:
"""
OVERVIEW
Basically our LISA dataset contains around 44k images and a bunch of labels

Labels are:
  - light status: {go, slow, stop}
  - bounding box of where the stoplight is xyxy coordinates (top left, bottom right)
  - each image may have many labels since there are many stoplights in a single image
"""

In [ ]:
"""
TODO: Define our model (YOLO)

what are the input dimensions?
what is the output shape?

"""